In [1]:
%%capture --no-stdout
%reload_ext watermark
%watermark -uniz --author "Prayson W. Daniel" -vm -p duckdb,polars,weaviate,ollama

Author: Prayson W. Daniel

Last updated: 2024-11-13T11:55:13.399498+01:00

Python implementation: CPython
Python version       : 3.11.10
IPython version      : 8.29.0

duckdb  : 1.1.3
polars  : 1.13.0
weaviate: 4.9.3
ollama  : not installed

Compiler    : Clang 15.0.0 (clang-1500.3.9.4)
OS          : Darwin
Release     : 23.5.0
Machine     : arm64
Processor   : arm
CPU cores   : 16
Architecture: 64bit



In [2]:
import weaviate
from weaviate.classes.config import Configure


In [3]:
client = weaviate.connect_to_local()
client.collections.delete_all()

{'message': 'Failed to send 5 objects in a batch of 5. Please inspect client.batch.failed_objects or collection.batch.failed_objects for the failed objects.'}


In [4]:

client.collections.create(
"Datax",
vectorizer_config=[
    Configure.NamedVectors.text2vec_ollama(
        name="ds_search",
        source_properties=["description"],
        api_endpoint="http://host.docker.internal:11434",
        model="nomic-embed-text", 
    )
],
 generative_config=Configure.Generative.ollama(
    api_endpoint="http://host.docker.internal:11434",  
    model="llama3.2"
)
)

In [5]:
objects = [
    {"title": "Shahnoza", "description": "Senior Data Scientist & Tech Lead | Machine Learning & MLOps"},
    {"title": "Mette", "description": "Data Scientist | AI & Machine Learning Engineer | Physicist"},
    {"title": "Ulrik", "description": "Senior Data Scientist @NTT Data / Machine Learning & MLOps / Biophysicist turned Data Scientist"},
    {"title": "Manuel", "description": "Data Scientist & AI Engineer @ NTT Data | MSc Computer Science"},
    {"title": "Nicolai", "description": "Lead AI Solution Architect @ NTT DATA | Teaching AI & ML @ CBS"},
]

In [7]:
collection = client.collections.get("DataX")

with collection.batch.dynamic() as batch:
    for obj in objects:
        weaviate_obj = {
            "title": obj["title"],
            "description": obj["description"],
        }


        batch.add_object(
            properties=weaviate_obj,
          
        )

In [9]:
# near_text | hybrid

response = collection.query.hybrid(
    query="Who are MLOps?",  
    limit=5,
    alpha=0.0
)

WeaviateQueryError: Query call with protocol GRPC search failed with message <AioRpcError of RPC that terminated with:
	status = StatusCode.UNKNOWN
	details = "could not find class DataX in schema"
	debug_error_string = "UNKNOWN:Error received from peer  {grpc_message:"could not find class DataX in schema", grpc_status:2, created_time:"2024-11-13T11:56:56.944999+01:00"}"
>.

In [ ]:
for obj in response.objects:
    print(f'{obj.properties["title"]} - {obj.properties["description"]}')